## Client

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import precision_recall_curve

import urllib.request
import json

In [2]:
def type_conversion(x):
    if type(x) == np.float64:
        return float(x)
    if type(x) == np.int64:
        return int(x)
    return x

def get_prediction(x, verbose=False):
    body = {feat[0]: feat[1] for feat in zip(['HAEMATOCRIT', 'HAEMOGLOBINS', 'ERYTHROCYTE', 'LEUCOCYTE',
                                    'THROMBOCYTE', 'MCH', 'MCHC', 'MCV', 'AGE', 'SEX'], x.apply(type_conversion))}
    
    myurl = 'http://127.0.0.1:5000/predict'
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')    
    req.add_header('Content-Length', len(jsondataasbytes))
    
    response = urllib.request.urlopen(req, jsondataasbytes)
    pred_data = json.loads(response.read())
    return pred_data if verbose else pred_data['predictions']

In [3]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

Запросим предсказания с одного объекта:

In [4]:
get_prediction(X_test.iloc[0, :], True)

{'AGE': [69],
 'ERYTHROCYTE': [4.56],
 'HAEMATOCRIT': [37.3],
 'HAEMOGLOBINS': [13.4],
 'LEUCOCYTE': [10.3],
 'MCH': [29.4],
 'MCHC': [35.9],
 'MCV': [81.8],
 'SEX': ['F'],
 'THROMBOCYTE': [211],
 'predictions': 0.2448779046535492,
 'success': True}

И со всей тестовой выборки:

In [5]:
preds = X_test.apply(get_prediction, axis=1)

In [6]:
preds[:10]

0    0.244878
1    0.048871
2    0.176370
3    0.501244
4    0.933302
5    0.325912
6    0.803829
7    0.210386
8    0.460991
9    0.388969
dtype: float64

In [7]:
preds.shape

(1103,)

In [8]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

print(f'Best Threshold = {thresholds[ix]}, F-Score = {fscore[ix]:.3f}, Precision = {precision[ix]:.3f}, Recall = {recall[ix]:.3f}')

Best Threshold = 0.25811153650283813, F-Score = 0.689, Precision = 0.590, Recall = 0.827
